# Import packages

In [1]:
from pathlib import Path
import os, sys
for p in [Path.cwd()] + list(Path.cwd().parents):
    if p.name == 'Multifirefly-Project':
        os.chdir(p)
        sys.path.insert(0, str(p / 'multiff_analysis/multiff_code/methods'))
        break

%load_ext autoreload
%autoreload 2


from data_wrangling import specific_utils, combine_info_utils
from pattern_discovery import pattern_by_trials, pattern_by_trials, cluster_analysis, organize_patterns_and_features, category_class
from decision_making_analysis.cluster_replacement import cluster_replacement_utils
from decision_making_analysis.decision_making import decision_making_class, decision_making_utils, intended_targets_classes
from decision_making_analysis.GUAT import GUAT_collect_info_class, GUAT_combine_info_class
from decision_making_analysis.compare_GUAT_and_TAFT import GUAT_vs_TAFT_class, GUAT_vs_TAFT_x_sessions_class, helper_GUAT_vs_TAFT_class
from visualization.matplotlib_tools import plot_trials, plot_behaviors_utils
from visualization.animation import animation_class
from null_behaviors import show_null_trajectory, find_best_arc, curvature_utils, curv_of_traj_utils
from machine_learning.ml_methods import regression_utils, classification_utils, prep_ml_data_utils, hyperparam_tuning_class
from visualization.plotly_polar_tools import plotly_utils_polar, plotly_for_ff_polar, plotly_for_trajectory_polar
from machine_learning.ml_methods import ml_methods_class
from visualization.dash_tools.dash_main_class_methods import dash_applied_to_GUAT_TAFT
from decision_making_analysis.advanced_modeling import model_choices

import os, sys
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from os.path import exists
import math
import copy
import matplotlib.pyplot as plt
import pandas as pd
import itertools
import matplotlib.pyplot as plt
import gc
from scipy import stats
from IPython.display import HTML
from matplotlib import rc
from sklearn.svm import SVC
from sklearn.ensemble import AdaBoostClassifier, BaggingClassifier
from sklearn.neural_network import MLPClassifier, MLPRegressor
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
import warnings
import os, sys, sys
from importlib import reload
from sklearn.exceptions import ConvergenceWarning


plt.rcParams["animation.html"] = "html5"
os.environ['KMP_DUPLICATE_LIB_OK']='True'
rc('animation', html='jshtml')
matplotlib.rcParams.update(matplotlib.rcParamsDefault)
matplotlib.rcParams['animation.embed_limit'] = 2**128
pd.set_option('display.float_format', lambda x: '%.5f' % x)
np.set_printoptions(suppress=True)
pd.options.display.max_rows = 50



Set up logging configuration.


# try funcs in class

https://chatgpt.com/c/68a3381b-f374-8324-b3c5-bdf77f515b21

In [ ]:
from decision_making_analysis.advanced_modeling.choice_model import *

In [ ]:
multiff_code/methods/decision_making_analysis/advanced_modeling/model_choices.py

## 1

In [ ]:
# Simulate
data = make_synthetic_dataset(num_decisions=800, E=4, D=6, K_cap=6)

# Split
idx = np.arange(len(data["ego"]))
np.random.shuffle(idx)
ntr = int(0.8*len(idx))
tr, va = idx[:ntr], idx[ntr:]

# Success logs
X = torch.tensor(data["succX"], dtype=torch.float32)
y = torch.tensor(data["succY"], dtype=torch.float32)
Xtr, ytr = X[tr], y[tr]; Xva, yva = X[va], y[va]

# DataLoaders for choice
tr_loader = DataLoader(DecisionDS(data, tr), batch_size=64, shuffle=True,  collate_fn=collate)
va_loader = DataLoader(DecisionDS(data, va), batch_size=64, shuffle=False, collate_fn=collate)

# Models
E = tr_loader.dataset.D["E"]
Ddim = tr_loader.dataset.D["D"]
sp = SuccessPredictor(in_dim=Ddim, hidden=64, p_drop=0.1)
cm = ContextAwareChoice(dim_ego=E, dim_retry=Ddim, dim_cand=Ddim, hidden=64)

# --- Train SuccessPredictor ---
opt1 = torch.optim.AdamW(sp.parameters(), lr=3e-3, weight_decay=1e-2)
for ep in range(5):
    sp.train(); opt1.zero_grad()
    p = sp(Xtr); loss = F.binary_cross_entropy(p, ytr)
    loss.backward(); opt1.step()
    with torch.no_grad():
        sp.eval()
        tr_bce = F.binary_cross_entropy(sp(Xtr), ytr).item()
        va_bce = F.binary_cross_entropy(sp(Xva), yva).item()
        tr_acc = ((sp(Xtr)>0.5).float()==ytr).float().mean().item()
        va_acc = ((sp(Xva)>0.5).float()==yva).float().mean().item()
    print(f"[Success] ep {ep+1} | train BCE {tr_bce:.3f} acc {tr_acc:.3f} | val BCE {va_bce:.3f} acc {va_acc:.3f}")

# --- Train Choice model ---
opt2 = torch.optim.AdamW(cm.parameters(), lr=3e-3, weight_decay=1e-2)

def run_epoch(loader, train=True):
    tot, n, correct = 0.0, 0, 0
    for b in loader:
        B,K,Dd = b["cand"].shape
        with torch.no_grad():
            pr = sp(b["retry"])                                           # (B,)
            pc = sp(b["cand"].reshape(B*K, Dd)).reshape(B,K) if K>0 else torch.zeros(B,0)
        scores, probs = cm(b["ego"], b["retry"], b["cand"], b["tcr"], b["tcc"],
                            b["ar"], b["ac"], pr, pc, b["mask"])
        loss = F.cross_entropy(scores, b["yidx"])
        if train:
            opt2.zero_grad(); loss.backward(); opt2.step()
        tot += loss.item()*B; n += B
        pm = probs.clone()
        if pm.shape[1]>1: pm[:,1:][ b["mask"] ] = 0.0
        correct += (pm.argmax(dim=1) == b["yidx"]).sum().item()
    return tot/max(1,n), correct/max(1,n)

for ep in range(8):
    tr_nll, tr_acc = run_epoch(tr_loader, True)
    va_nll, va_acc = run_epoch(va_loader, False)
    print(f"[Choice ] ep {ep+1} | train NLL {tr_nll:.3f} acc {tr_acc:.3f} | val NLL {va_nll:.3f} acc {va_acc:.3f}")

# --- Show a few predicted probabilities (masking pads) ---
b = next(iter(va_loader))
B,K,Dd = b["cand"].shape
with torch.no_grad():
    pr = sp(b["retry"]); pc = sp(b["cand"].reshape(B*K, Dd)).reshape(B,K) if K>0 else torch.zeros(B,0)
    _, probs = cm(b["ego"], b["retry"], b["cand"], b["tcr"], b["tcc"], b["ar"], b["ac"], pr, pc, b["mask"])
    if probs.shape[1]>1: probs[:,1:][ b["mask"] ] = 0.0
print("\nSample predicted probs for first 3 items (col 0=retry, cols 1..K=candidates):")
print(probs[:3].numpy())


## 2

In [ ]:

# ----- A) Pretrain success predictor on attempt logs -----
# x_attempt: (N, D_succ)  features at attempt time (retry/candidate attempts pooled)
# y_attempt: (N,)         0/1 labels for "stopped inside boundary?"
pred = success_model(x_attempt)                      # (N,)
loss_succ = F.binary_cross_entropy(pred, y_attempt.float())
loss_succ.backward(); opt_succ.step(); opt_succ.zero_grad()

# ----- B) Choice model training at decision points -----
# Batch dict (from collate) with shapes noted above:
batch = collate_retry_switch(list_of_examples)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
for k in batch: batch[k] = batch[k].to(device)

# Split features for success vs choice if you keep separate featurizations;
# here I reuse the same (retry_feat / cand_feat) for illustration:
retry_feat_succ = batch["retry_feat"]                 # (B, R_succ)
cand_feat_succ  = batch["cand_feat"]                  # (B, K, C_succ)

# Compute success probabilities per option (vectorized over candidates):
with torch.no_grad():  # keep SuccessPredictor frozen if you pretrain it
    p_retry = success_model(retry_feat_succ)          # (B,)
    B, K, Csucc = cand_feat_succ.shape
    p_cand  = success_model(cand_feat_succ.reshape(B*K, Csucc)).reshape(B, K)  # (B, K)

# Run the choice model to get scores/probs over {retry} ∪ {candidates}
scores, probs = choice_model(
    ego             = batch["ego"],                   # (B, E)
    retry_feat      = batch["retry_feat"],            # (B, R)
    cand_feat       = batch["cand_feat"],             # (B, K, C)
    time_cost_retry = batch["time_cost_retry"],       # (B,)
    time_cost_cand  = batch["time_cost_cand"],        # (B, K)
    attempts_retry  = batch["attempts_retry"],        # (B,)
    attempts_cand   = batch["attempts_cand"],         # (B, K)
    p_succ_retry    = p_retry,                        # (B,)
    p_succ_cand     = p_cand,                         # (B, K)
    mask            = batch["cand_mask"]              # (B, K) True=PAD
)

# Cross-entropy over K+1 columns (0=retry, 1..K=candidates)
loss_choice = F.cross_entropy(scores, batch["chosen_idx"])  # scores are logits
loss_choice.backward(); opt_choice.step(); opt_choice.zero_grad()

# Optional: for metrics (argmax), zero out padded columns first
probs_masked = probs.clone()
probs_masked[:, 1:][ batch["cand_mask"] ] = 0.0
pred_idx = probs_masked.argmax(dim=1)  # (B,) predicted option index


# smoke test

In [2]:
import math
from dataclasses import dataclass
from typing import Dict, List, Optional, Tuple

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader



In [4]:
torch.manual_seed(0)

# 3A) Success head on random data
Xs = torch.randn(512, 10)
ys = torch.bernoulli(torch.full((512,), 0.5))
ds = model_choices.SuccessDataset(Xs, ys)
dl = DataLoader(ds, batch_size=64, shuffle=True)
succ = model_choices.SuccessPredictor(in_dim=10)
model_choices.train_success_head(succ, dl, epochs=2)

# 3B) Choice model with masking
items = []
for _ in range(256):
    K = torch.randint(low=2, high=6, size=(1,)).item()  # 2..5 options
    option_feats = torch.randn(K, 8)
    p_succ_opt = torch.sigmoid(torch.randn(K))
    extra_costs = torch.randn(K, 2)
    chosen = torch.randint(low=0, high=K, size=(1,)).item()
    items.append({
        "option_features": option_feats,
        "p_succ_opt": p_succ_opt,
        "extra_costs": extra_costs,
        "option_mask": torch.ones(K, dtype=torch.bool),
        "chosen_index": torch.tensor(chosen),
    })
dc = model_choices.ChoiceDataset(items)
dlc = DataLoader(dc, batch_size=32, shuffle=True, collate_fn=model_choices.choice_collate)
chooser = model_choices.ChoiceScorer(option_dim=8, extra_cost_dim=2, use_psucc=True)
model_choices.train_choice_model(chooser, dlc, epochs=2)

print("Smoke test complete.")

[Success] epoch 1: loss=0.6924
[Success] epoch 2: loss=0.6911
[Choice] epoch 1: loss=1.1762
[Choice] epoch 2: loss=1.1666
Smoke test complete.


In [5]:
model_choices

<module 'decision_making_analysis.advanced_modeling.model_choices' from '/Users/dusiyi/Documents/Multifirefly-Project/multiff_analysis/multiff_code/methods/decision_making_analysis/advanced_modeling/model_choices.py'>

# toy data

In [7]:
# ---- 3A: success head toy data ----
import torch
from torch.utils.data import DataLoader

X_stop = torch.tensor([
    [ 0.2, -0.3, 0.1],   # stop features (D_s = 3)
    [ 1.0,  0.4, 0.5],
    [-0.7,  0.2, 0.9],
    [ 0.3,  0.1, 0.2],
], dtype=torch.float32)
y_inside = torch.tensor([1., 1., 0., 1.], dtype=torch.float32)

ds = SuccessDataset(X_stop, y_inside)
dl = DataLoader(ds, batch_size=2, shuffle=True)

model = SuccessPredictor(in_dim=3)
train_success_head(model, dl, epochs=3, lr=1e-3)

with torch.no_grad():
    print("p_hat_succ:", model(X_stop).tolist())


[Success] epoch 1: loss=0.7340
[Success] epoch 2: loss=0.7292
[Success] epoch 3: loss=0.7245
p_hat_succ: [0.46155235171318054, 0.4903356432914734, 0.45318105816841125, 0.45039984583854675]


In [8]:
# ---- 3B: choice model toy batch (variable K) ----
items = [
    {   # episode 1: K=3 options (e.g., {retry, ff#7, ff#12})
        "option_features": torch.tensor([[0.1, 0.2], [-0.3, 0.5], [0.4, -0.1]], dtype=torch.float32),  # [3, D_o=2]
        "p_succ_opt":     torch.tensor([0.70, 0.25, 0.55], dtype=torch.float32),                       # [3]
        "extra_costs":    torch.tensor([[0.9, 0.1], [0.2, 0.7], [0.6, 0.3]], dtype=torch.float32),     # [3, D_c=2]
        "chosen_index":   torch.tensor(0),  # picked the first option
    },
    {   # episode 2: K=2 options
        "option_features": torch.tensor([[0.0, -0.2], [0.8, 0.1]], dtype=torch.float32),               # [2, 2]
        "p_succ_opt":     torch.tensor([0.40, 0.65], dtype=torch.float32),                              # [2]
        "extra_costs":    torch.tensor([[0.3, 0.4], [0.1, 0.2]], dtype=torch.float32),                 # [2, 2]
        "chosen_index":   torch.tensor(1),
    },
]

loader = DataLoader(ChoiceDataset(items), batch_size=2, shuffle=True, collate_fn=choice_collate)
chooser = ChoiceScorer(option_dim=2, extra_cost_dim=2, use_psucc=True)

for _ in range(3):
    for batch in loader:
        logits = chooser(batch["option_features"], batch["mask"], batch["p_succ_opt"], batch["extra_costs"])
        loss = masked_cross_entropy(logits, batch["targets"], batch["mask"])
        chooser.zero_grad(); loss.backward()
        for p in chooser.parameters(): 
            if p.grad is not None:
                p.data -= 1e-2 * p.grad   # tiny manual SGD step
    print("train loss:", float(loss))


train loss: 0.875237226486206
train loss: 0.8737525939941406
train loss: 0.8722797632217407


# synthetic data

In [13]:
torch.manual_seed(0)
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")
# 1) Train success head on synthetic stops
Xtr, ytr, Xva, yva = generate_synthetic_success_data(N=5000)
succ = SuccessPredictor(in_dim=6)
dl_tr = DataLoader(SuccessDataset(Xtr, ytr), batch_size=128, shuffle=True)
train_success_head(succ, dl_tr, epochs=5, lr=1e-3, device=device)
evaluate_success_head(succ.to(device), Xva.to(device), yva.to(device))

# 2) Build a synthetic choice dataset that uses p_succ from the success head
items = build_choice_items(succ, N=3000)
ntr = int(0.85 * len(items))
dc_tr = ChoiceDataset(items[:ntr])
dc_va = ChoiceDataset(items[ntr:])
dl_tr_c = DataLoader(dc_tr, batch_size=64, shuffle=True, collate_fn=choice_collate)
dl_va_c = DataLoader(dc_va, batch_size=128, shuffle=False, collate_fn=choice_collate)

chooser = ChoiceScorer(option_dim=2, extra_cost_dim=2, use_psucc=True)
train_choice_model(chooser, dl_tr_c, epochs=5, lr=1e-3, device=device)
acc = eval_choice_top1(chooser.to(device), dl_va_c, device=device)
print(f"[Choice/Val] top-1 accuracy={acc:.3f}")

Using device: cpu
[Success] epoch 1: loss=0.5714
[Success] epoch 2: loss=0.4938
[Success] epoch 3: loss=0.4550
[Success] epoch 4: loss=0.4413
[Success] epoch 5: loss=0.4312
[Success/Val] acc=0.812  corr(p, -miss)=0.38  corr(p, bright)=-0.01  corr(p, align)=0.88
[Choice] epoch 1: loss=1.0445
[Choice] epoch 2: loss=0.8030
[Choice] epoch 3: loss=0.7667
[Choice] epoch 4: loss=0.7603
[Choice] epoch 5: loss=0.7538
[Choice/Val] top-1 accuracy=0.704


# functions

In [ ]:
"""
Tiny PyTorch skeleton for MultiFF retry/switch modeling.

Implements the two core training steps and leaves hooks for the optional
variants. The code is intentionally lightweight: small MLPs, masking for
variable option counts, and minimal training loops.

You can paste this into a file and adapt the Dataset stubs to your
preprocessing.
"""
from __future__ import annotations

import math
from dataclasses import dataclass
from typing import Dict, List, Optional, Tuple

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader


# -------------------------------
# Utilities
# -------------------------------

def mlp(in_dim: int, hidden: List[int], out_dim: int, dropout: float = 0.0) -> nn.Sequential:
    layers: List[nn.Module] = []
    last = in_dim
    for h in hidden:
        layers += [nn.Linear(last, h), nn.ReLU()]  # keep it simple
        if dropout > 0:
            layers += [nn.Dropout(dropout)]
        last = h
    layers += [nn.Linear(last, out_dim)]
    return nn.Sequential(*layers)


def masked_cross_entropy(logits: torch.Tensor, targets: torch.Tensor, mask: torch.Tensor) -> torch.Tensor:
    """
    Cross-entropy for variable option sets.
    logits: [B, Kmax]
    targets: [B] (index of chosen option in 0..K-1)
    mask: [B, Kmax] boolean (True for valid options)
    """
    # Put -inf on invalid options so softmax ignores them
    logits_masked = logits.masked_fill(~mask, -1e9)
    return F.cross_entropy(logits_masked, targets)


# -------------------------------
# 3A) Capture-success predictor
# -------------------------------

class SuccessPredictor(nn.Module):
    """
    Predicts p_succ = P(next stop is inside boundary for that target).

    Inputs: stop_features: [B, D_s]
    Output: p_succ: [B]
    """
    def __init__(self, in_dim: int, hidden: List[int] = [32]):
        super().__init__()
        self.net = mlp(in_dim, hidden, 1, dropout=0.0)

    def forward(self, stop_features: torch.Tensor) -> torch.Tensor:
        return torch.sigmoid(self.net(stop_features)).squeeze(-1)


# -------------------------------
# 3B) Choice scorer over {retry} ∪ {other targets}
# -------------------------------

class ChoiceScorer(nn.Module):
    """
    Scores each candidate option with a utility U_i(t) and produces logits.

    Forward inputs:
      - option_features: [B, Kmax, D_o]
      - mask: [B, Kmax] boolean (True for valid candidates)
      - p_succ_opt: Optional[Tensor] [B, Kmax] (per-option success proba)
      - extra_costs: Optional[Tensor] [B, Kmax, D_c] (e.g., time-to-go, turn cost)

    Returns:
      - logits: [B, Kmax]
    """
    def __init__(self, option_dim: int, extra_cost_dim: int = 0, use_psucc: bool = True,
                 hidden: List[int] = [64, 32]):
        super().__init__()
        self.use_psucc = use_psucc
        in_dim = option_dim + extra_cost_dim + (1 if use_psucc else 0)
        self.net = mlp(in_dim, hidden, 1, dropout=0.0)

    def forward(
        self,
        option_features: torch.Tensor,
        mask: torch.Tensor,
        p_succ_opt: Optional[torch.Tensor] = None,
        extra_costs: Optional[torch.Tensor] = None,
    ) -> torch.Tensor:
        B, Kmax, D_o = option_features.shape
        feats = [option_features]
        if self.use_psucc:
            if p_succ_opt is None:
                raise ValueError("p_succ_opt must be provided when use_psucc=True")
            feats.append(p_succ_opt.unsqueeze(-1))  # [B, Kmax, 1]
        if extra_costs is not None:
            feats.append(extra_costs)
        X = torch.cat(feats, dim=-1)  # [B, Kmax, Din]
        logits = self.net(X).squeeze(-1)  # [B, Kmax]
        # Mask invalid options with -inf to avoid accidental selection
        logits = logits.masked_fill(~mask, -1e9)
        return logits


# -------------------------------
# Optional variants (4)
# -------------------------------

class RetrySwitchHead(nn.Module):
    """Binary logit: retry vs switch after a near-miss."""
    def __init__(self, in_dim: int, hidden: List[int] = [32]):
        super().__init__()
        self.net = mlp(in_dim, hidden, 1)

    def forward(self, features: torch.Tensor) -> torch.Tensor:
        # returns P(retry)
        return torch.sigmoid(self.net(features)).squeeze(-1)


class TwoStagePolicy(nn.Module):
    """
    Stage 1: binary retry vs switch (on near-miss features).
    Stage 2: if switch, choose among other targets using ChoiceScorer.
    """
    def __init__(self, retry_in_dim: int, option_dim: int, extra_cost_dim: int = 0, use_psucc: bool = True):
        super().__init__()
        self.retry_head = RetrySwitchHead(retry_in_dim)
        self.choice = ChoiceScorer(option_dim, extra_cost_dim, use_psucc)

    def forward(self, retry_features: torch.Tensor,
                option_features: torch.Tensor, mask: torch.Tensor,
                p_succ_opt: Optional[torch.Tensor] = None,
                extra_costs: Optional[torch.Tensor] = None) -> Tuple[torch.Tensor, torch.Tensor]:
        p_retry = self.retry_head(retry_features)
        logits_switch = self.choice(option_features, mask, p_succ_opt, extra_costs)
        return p_retry, logits_switch


class HazardHead(nn.Module):
    """
    Discrete-time hazard after a near-miss: h_t = P(switch at t | not switched yet).

    Input per step features X_t -> h_t via sigmoid(MLP).
    Loss implemented via discrete-time survival likelihood.
    """
    def __init__(self, in_dim: int, hidden: List[int] = [32]):
        super().__init__()
        self.net = mlp(in_dim, hidden, 1)

    def forward(self, step_features: torch.Tensor, step_mask: torch.Tensor) -> torch.Tensor:
        """
        step_features: [B, T, D]
        step_mask: [B, T] (True for valid time steps)
        Returns hazards h_t in [0,1]: [B, T]
        """
        h = torch.sigmoid(self.net(step_features)).squeeze(-1)
        return h * step_mask.float()

    @staticmethod
    def survival_nll(h: torch.Tensor, event_index: torch.Tensor, step_mask: torch.Tensor) -> torch.Tensor:
        """
        Negative log-likelihood for discrete-time hazards.
        h: [B, T] hazards
        event_index: [B] index of switch time; if censored (no switch in window), set to -1
        step_mask: [B, T]
        """
        # log S_{t} = sum_{k < t} log(1 - h_k);  log f(t) = log S_t + log h_t
        eps = 1e-6
        log1m_h = torch.log(torch.clamp(1 - h, min=eps)) * step_mask
        cumsums = torch.cumsum(log1m_h, dim=1)  # [B, T]
        B, T = h.shape
        nll = []
        for b in range(B):
            t_star = event_index[b].item()
            if t_star >= 0:  # observed switch
                surv = cumsums[b, t_star - 1] if t_star > 0 else torch.tensor(0.0, device=h.device)
                log_h = torch.log(torch.clamp(h[b, t_star], min=eps))
                nll.append(-(surv + log_h))
            else:  # censored at last valid step
                last = int(step_mask[b].nonzero(as_tuple=False)[-1])
                surv = cumsums[b, last]
                nll.append(-surv)
        return torch.stack(nll).mean()


# -------------------------------
# Belief / POMDP-ish helper (very simple)
# -------------------------------

@dataclass
class BeliefState:
    alpha: torch.Tensor  # evidence for success
    beta: torch.Tensor   # evidence for failure

    def p_succ(self) -> torch.Tensor:
        return self.alpha / (self.alpha + self.beta + 1e-9)


def update_belief(
    belief: BeliefState,
    flash_strength: torch.Tensor,
    miss_distance: Optional[torch.Tensor] = None,
    decay: float = 0.95,
) -> BeliefState:
    """
    Tiny heuristic updater: decay old evidence, add flash as positive evidence,
    add miss_distance (scaled) as negative evidence.
    """
    alpha = decay * belief.alpha + flash_strength
    if miss_distance is not None:
        beta = decay * belief.beta + miss_distance
    else:
        beta = decay * belief.beta
    return BeliefState(alpha=alpha, beta=beta)


# -------------------------------
# Datasets (stubs you will replace)
# -------------------------------

class SuccessDataset(Dataset):
    """Each item: (stop_features [D_s], label {0,1})"""
    def __init__(self, X: torch.Tensor, y: torch.Tensor):
        assert X.ndim == 2 and y.ndim == 1
        self.X, self.y = X.float(), y.long()

    def __len__(self) -> int: return len(self.y)

    def __getitem__(self, i: int) -> Tuple[torch.Tensor, torch.Tensor]:
        return self.X[i], self.y[i].float()


class ChoiceDataset(Dataset):
    """
    Each item is a dict with keys:
      - option_features: [K, D_o]
      - option_mask: [K] (bool)
      - chosen_index: int in [0, K-1]
      - p_succ_opt: Optional [K]
      - extra_costs: Optional [K, D_c]
    """
    def __init__(self, items: List[Dict[str, torch.Tensor]]):
        self.items = items

    def __len__(self): return len(self.items)

    def __getitem__(self, i: int) -> Dict[str, torch.Tensor]:
        return self.items[i]


def choice_collate(batch: List[Dict[str, torch.Tensor]]) -> Dict[str, torch.Tensor]:
    Kmax = max(item["option_features"].shape[0] for item in batch)
    B = len(batch)
    D_o = batch[0]["option_features"].shape[1]
    option_features = torch.zeros(B, Kmax, D_o)
    mask = torch.zeros(B, Kmax, dtype=torch.bool)
    targets = torch.zeros(B, dtype=torch.long)
    p_succ_opt = None
    extra_costs = None

    has_ps = all("p_succ_opt" in item for item in batch)
    has_ec = all("extra_costs" in item for item in batch)

    if has_ps:
        p_succ_opt = torch.zeros(B, Kmax)
    if has_ec:
        D_c = batch[0]["extra_costs"].shape[1]
        extra_costs = torch.zeros(B, Kmax, D_c)

    for b, item in enumerate(batch):
        K = item["option_features"].shape[0]
        option_features[b, :K] = item["option_features"]
        mask[b, :K] = True
        targets[b] = int(item["chosen_index"])  # ensure within 0..K-1
        if has_ps:
            p_succ_opt[b, :K] = item["p_succ_opt"]
        if has_ec:
            extra_costs[b, :K] = item["extra_costs"]

    out = {"option_features": option_features, "mask": mask, "targets": targets}
    if has_ps:
        out["p_succ_opt"] = p_succ_opt
    if has_ec:
        out["extra_costs"] = extra_costs
    return out


# -------------------------------
# Training loops (minimal)
# -------------------------------

def train_success_head(model: SuccessPredictor, loader: DataLoader, epochs: int = 10, lr: float = 1e-3,
                       device: str = "cpu") -> None:
    model.to(device)
    opt = torch.optim.AdamW(model.parameters(), lr=lr)
    bce = nn.BCELoss()
    model.train()
    for ep in range(epochs):
        total = 0.0
        for X, y in loader:
            X, y = X.to(device), y.to(device)
            p = model(X)
            loss = bce(p, y)
            opt.zero_grad(); loss.backward(); opt.step()
            total += loss.item() * X.size(0)
        print(f"[Success] epoch {ep+1}: loss={total/len(loader.dataset):.4f}")


def train_choice_model(model: ChoiceScorer, loader: DataLoader, epochs: int = 10, lr: float = 1e-3,
                       device: str = "cpu") -> None:
    model.to(device)
    opt = torch.optim.AdamW(model.parameters(), lr=lr)
    model.train()
    for ep in range(epochs):
        total = 0.0
        for batch in loader:
            opt.zero_grad()
            option_features = batch["option_features"].to(device)
            mask = batch["mask"].to(device)
            targets = batch["targets"].to(device)
            p_succ_opt = batch.get("p_succ_opt")
            extra_costs = batch.get("extra_costs")
            if p_succ_opt is not None: p_succ_opt = p_succ_opt.to(device)
            if extra_costs is not None: extra_costs = extra_costs.to(device)

            logits = model(option_features, mask, p_succ_opt, extra_costs)
            loss = masked_cross_entropy(logits, targets, mask)
            loss.backward(); opt.step()
            total += loss.item() * option_features.size(0)
        print(f"[Choice] epoch {ep+1}: loss={total/len(loader.dataset):.4f}")




[Success] epoch 1: loss=0.6924
[Success] epoch 2: loss=0.6911
[Choice] epoch 1: loss=1.1762
[Choice] epoch 2: loss=1.1666
Smoke test complete.
